In [14]:
from tqdm import tqdm
from glob import glob
import json
import random

In [15]:
def rebel_format(triplets):
    """
    Convert
    [['Bruno Santana', 'participant of', '2004 Summer Olympics'],
    ['Bruno Santana', 'participant of', '2008 Summer Olympics'],
    ['Bruno Santana', 'country of citizenship', 'Brazil']]
    to rebel format,
    <triplet> Bruno Santana <subj> 2004 Summer Olympics <obj> participant of <subj> 2008 Summer Olympics <obj> participant of <subj> Brazil <obj> country of citizenship
    """
    q = []
    for no, triple in enumerate(triplets):
        obj = ['<obj>'] + triple[1].split()
        subj = ['<subj>'] + triple[2].split()
        if no > 0 and triple[0] == triplets[no - 1][0]:
            q.extend(subj + obj)
        else:
            triplet = ['<triplet>'] + triple[0].split()
            q.extend(triplet + subj + obj)
    return ' '.join(q)

def parse_rebel(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace('<s>', '').replace("<pad>", '').replace('</s>', '').split():
        if token == '<triplet>':
            current = 't'
            if relation != '':
                triplets.append(
                    {'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == '<subj>':
            current = 's'
            if relation != '':
                triplets.append(
                    {'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
            object_ = ''
        elif token == '<obj>':
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(),
                         'type': relation.strip(),
                         'tail': object_.strip()})
    return triplets

In [16]:
files = glob('processed-kg-*.jsonl')
files

['processed-kg-astroawani.jsonl', 'processed-kg-wikipedia.jsonl']

In [17]:
with open('train.json', 'w') as fopen_jsonl:
    for f in files:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                d = {"translation": {"src": l['text'], "tgt": l['kg'], 'prefix': 'teks ke grafik pengetahuan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                fopen_jsonl.flush()

In [18]:
!head -n 3 train.json

{"translation": {"src": "Padah jalin hubungan sulit dengan pekerja sendiri, CEO McDonald's dipecat serta merta", "tgt": "<triplet> Padah <subj> hubungan sulit <obj> mempunyai <triplet> hubungan sulit <subj> pekerja sendiri <obj> dengan <triplet> Padah <subj> CEO McDonald's <obj> dipecat", "prefix": "teks ke grafik pengetahuan: "}}
{"translation": {"src": "CEO tidak boleh menjalin hubungan dengan mana-mana kakitangan.", "tgt": "<triplet> CEO <subj> kakitangan <obj> tidak boleh menjalin hubungan dengan", "prefix": "teks ke grafik pengetahuan: "}}
{"translation": {"src": "SYARIKAT rantaian makanan segera terkemuka dunia, McDonald's Corp mengesahkan telah memecat Ketua Pegawai Eksekutif (CEO), Steve Easterbrook selepas menjalinkan hubungan sulit dengan salah seorang kakitangannya.", "tgt": "<triplet> <subj> yang telah memecat Steve Easterbrook <obj> mengesahkan <triplet> Steve Easterbrook <subj> hubungan yang tidak sesuai dengan pekerja <obj> telah", "prefix": "teks ke grafik pengetahuan: 

In [14]:
!shuf train.json > shuf-train.json

In [19]:
!wc -l train.json

190102 train.json


In [16]:
data = []
with open('shuf-train.json') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if len(l['translation']['tgt'].split()) > 100:
            data.append(l)
        
len(data)

2532062it [00:08, 295700.23it/s]


65582

In [17]:
data[-2]

{'translation': {'src': 'Dameshkaft-e Kalat (, also Romanized as Dameshkaft-e Kalāt; also known as Dameshkaft) is a village in Doshman Ziari Rural District, in the Central District of Kohgiluyeh County, Kohgiluyeh and Boyer-Ahmad Province, Iran. At the 2006 census, its population was 151, in 30 families.',
  'tgt': '<triplet> Dameshkaft-e Kalat <subj> Daerah Luar Bandar Doshman Ziari <obj> terletak dalam entiti wilayah pentadbiran <triplet> Daerah Luar Bandar Doshman Ziari <subj> Daerah Tengah <obj> terletak dalam entiti wilayah pentadbiran <subj> Daerah Kohgiluyeh <obj> sebahagian daripada <subj> Iran <obj> negara <triplet> Daerah Tengah <subj> Daerah Kohgiluyeh <obj> terletak dalam entiti wilayah pentadbiran <subj> Wilayah Kohgiluyeh dan Boyer-Ahmad <obj> terletak dalam entiti wilayah pentadbiran <subj> Iran <obj> negara <triplet> Daerah Kohgiluyeh <subj> Daerah Tengah <obj> mengandungi entiti wilayah pentadbiran <subj> Wilayah Kohgiluyeh dan Boyer-Ahmad <obj> terletak dalam entiti w

In [18]:
parse_rebel(data[-2]['translation']['tgt'])

[{'head': 'Dameshkaft-e Kalat',
  'type': 'terletak dalam entiti wilayah pentadbiran',
  'tail': 'Daerah Luar Bandar Doshman Ziari'},
 {'head': 'Daerah Luar Bandar Doshman Ziari',
  'type': 'terletak dalam entiti wilayah pentadbiran',
  'tail': 'Daerah Tengah'},
 {'head': 'Daerah Luar Bandar Doshman Ziari',
  'type': 'sebahagian daripada',
  'tail': 'Daerah Kohgiluyeh'},
 {'head': 'Daerah Luar Bandar Doshman Ziari',
  'type': 'negara',
  'tail': 'Iran'},
 {'head': 'Daerah Tengah',
  'type': 'terletak dalam entiti wilayah pentadbiran',
  'tail': 'Daerah Kohgiluyeh'},
 {'head': 'Daerah Tengah',
  'type': 'terletak dalam entiti wilayah pentadbiran',
  'tail': 'Wilayah Kohgiluyeh dan Boyer-Ahmad'},
 {'head': 'Daerah Tengah', 'type': 'negara', 'tail': 'Iran'},
 {'head': 'Daerah Kohgiluyeh',
  'type': 'mengandungi entiti wilayah pentadbiran',
  'tail': 'Daerah Tengah'},
 {'head': 'Daerah Kohgiluyeh',
  'type': 'terletak dalam entiti wilayah pentadbiran',
  'tail': 'Wilayah Kohgiluyeh dan Boy

In [69]:
!head -n 3 shuf-train.json

{"translation": {"src": "Vladimir Luxuria (lahir 24 Jun 1965 di Foggia, Apulia) ialah seorang pelakon, penulis, ahli politik dan hos televisyen trans Itali. Luxuria ialah ahli parlimen Parti Kebangkitan Komunis, kepunyaan gabungan Kesatuan yang diketuai oleh Romano Prodi.\n\nBeliau adalah ahli Parlimen transgender terbuka pertama di Eropah, dan ahli parlimen transgender terbuka kedua di dunia selepas warga New Zealand Georgina Beyer. Dia kehilangan kerusinya dalam pilihan raya April 2008.\n\nDalam pilihan raya umum 2006, Luxuria telah dipilih ke Dewan Perwakilan oleh kawasan Lazio 1 di Rom. Dia kehilangan kerusinya dalam pilihan raya 2008. Selepas persaraan Beyer dan Luxuria, tiada ahli parlimen transgender dilaporkan di dunia, sehingga 2011, apabila Anna Grodzka dipilih ke parlimen Poland.", "tgt": "<triplet> Vladimir Luxuria <subj> Foggia <obj> tempat kelahiran <subj> hos televisyen <obj> pekerjaan <subj> 24 Jun 1965 <obj> tarikh lahir <subj> Parti Pemulihan Komunis <obj> ahli parti 

In [70]:
test_file = '/home/husein/ssd3/ctranslate2/en_test.jsonl.translated'

In [71]:
with open('test.json', 'w') as fopen_jsonl:
    with open(test_file) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if not l['triples_ms']:
                continue
                
            if not l['text_ms']:
                continue

            if len(l['text_ms'].split()) > 400:
                continue

            triples = []
            for t in l['triples_ms']:
                if not len(t['head']):
                    continue

                if not len(t['type']):
                    continue

                if not len(t['tail']):
                    continue

                triples.append([t['head'], t['type'], t['tail']])

            if not len(triples):
                continue

            right = rebel_format(triples)
            
            if len(right.split()) > 200:
                continue
                
            left = l['text_ms'].strip()
            left_en = l['text'].strip()
            
            if len(left) and len(right) and len(left.split()) < 1536 and len(right.split()) < 1536:
                d = {"translation": {"src": left, "tgt": right, 'prefix': 'teks ke grafik pengetahuan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                fopen_jsonl.flush()
                
            if random.random() > 0.5 and len(left_en) and len(right) and len(left_en.split()) < 1536 and len(right.split()) < 1536:
                d = {"translation": {"src": left_en, "tgt": right, 'prefix': 'teks ke grafik pengetahuan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                fopen_jsonl.flush()

152836it [00:05, 28895.53it/s]


In [72]:
!shuf test.json > shuf-test.json

In [15]:
!head -n 4000 test.json > test-4k.json

In [3]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-small-malaysian-cased')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
raw_datasets = load_dataset(
    'json',
    data_files='test-4k.json',
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/husein/.cache/huggingface/datasets/json/default-d5f4b4fd406d1504/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
raw_datasets['train'][0]

{'translation': {'src': 'Istana Beardslee ialah sebuah istana tiga tingkat di Little Falls, New York, Amerika Syarikat, dibina pada tahun 1860 sebagai replika istana Ireland, dan kini digunakan sebagai restoran. Manor itu telah dibina semula dua kali, selepas dibakar oleh kebakaran pada tahun 1919 dan 1989.\n\nIstana Beardslee ialah projek Lavina (Pardee) Beardslee yang balu, bermula pada tahun-tahun terakhirnya sekitar akhir 1790-an. Anak lelakinya, Augustus Beardslee, dikreditkan dengan bangunan Istana, walaupun cucu Lavina, Kapten Guy Roosevelt Beardslee (yang dilahirkan di rumah agam), yang menyelia siapnya projek sekitar tahun 1860. Kapten Beardslee menjalankan projek perniagaan yang memperoleh elektrik dan penjana dengan sistem pengedaran memulakan loji kuasa hidro tempatan yang kecil. Syarikat kecil ini berjaya masuk ke dalam Niagara Mohawk Power Corporation.',
  'tgt': '<triplet> Beardslee Castle <subj> USA <obj> country <subj> restaurant <obj> contoh <triplet> Little Falls, Ne

In [24]:
source_lang = 'src'
target_lang = 'tgt'
padding = True

def preprocess_function(examples):
    inputs = [ex['prefix'] + ex[source_lang] +
              tokenizer.eos_token for ex in examples["translation"]]
    targets = [ex[target_lang] + tokenizer.eos_token for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        padding=True,
        truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=1024,
            padding=True,
            truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100)
                                for l in label] for label in labels["input_ids"]]

    model_inputs["labels"] = labels["input_ids"]
    model_inputs.pop('token_type_ids', None)
    return model_inputs

In [25]:
train_dataset = raw_datasets['train'].map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [27]:
train_dataset

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})

In [23]:
train_dataset

Dataset({
    features: ['translation', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})